# EEG - Flow

## 4. ICA select and apply
Saves the ICA with the reviewer in the filename

Last edit: 17.04.2023 19:06
@anguyen

In [ ]:
import os
from datetime import datetime

from mne import pick_types, read_annotations
from mne.io import read_info, read_raw_fif
from mne.io.constants import FIFF
from mne.preprocessing import ICA, read_ica
from mne.viz import set_browser_backend

from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files

%matplotlib qt
set_browser_backend('qt')

_, DERIVATIVES_FOLDER_ROOT, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANT = 19        # int
GROUP       = 6         # int [1, 2, 3, 4, 5, 6, 7, 8]
TASK        = "oddball" # str [oddball, UT]
RUN         = 2         # int [1, 2]
PAST_REVIEWER = ""  # If empty, use EXPERIMENTER to write new files
                    # Else, if a name is provided, this will only load their
                    # review and not write anything
        
# if False, loads the fresh unreviewed ICA components of REVIEWER
LOAD_EXISTING_REVIEW = False
###############################################################################

derivatives_folder = get_folder(DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP)
fname_stem = get_fname(PARTICIPANT, GROUP, TASK, RUN)

# create derivatives ica plots subfolder
os.makedirs(derivatives_folder / fname_stem / "plots" / "ica", exist_ok=True)

# create locks
derivatives = [
    derivatives_folder / fname_stem / (fname_stem
                                       + "_step4_reviewed-1st-"
                                       + experimenter
                                       + "-ica.fif"
                                      ),
    derivatives_folder / fname_stem / (fname_stem 
                                       + "_step4_reviewed-2nd-"
                                       + experimenter
                                       + "-ica.fif"),
]

locks = lock_files(*derivatives)

# load previous steps
## load raw recording
raw = read_raw_fif(
    derivatives_folder / fname_stem / (fname_stem + "_step1_raw.fif"),
    preload=True
)
## load following annots
info = read_info(
    derivatives_folder / fname_stem / (fname_stem + "_step2_info.fif")
)
annot = read_annotations(
    derivatives_folder / fname_stem / (fname_stem
                                       + "_step2_oddball_with_bads_annot.fif")
)

# merge info and annots into current raw
raw.info["bads"] = info["bads"]
raw.set_annotations(annot)

# load ICAs
if not PAST_REVIEWER:
# if not LOAD_EXISTING_REVIEW:
    fname_ica1 = derivatives_folder / fname_stem / (fname_stem
                                                    + "_step3_1st-ica.fif")
    fname_ica2 = derivatives_folder / fname_stem / (fname_stem
                                                    + "_step3_2nd-ica.fif")
else:
    experimenter = PAST_REVIEWER
    fname_ica1 = derivatives_folder / fname_stem / (
                                                fname_stem
                                                + "_step4_reviewed-1st-"
                                                + experimenter +"-ica.fif")
    fname_ica2 = derivatives_folder / fname_stem / (
                                                fname_stem
                                                + "_step4_reviewed-2nd-"
                                                + experimenter + "-ica.fif")

ica1 = read_ica(fname_ica1)
ica2 = read_ica(fname_ica2)

In [ ]:
# Filter to final BP (1, 40) Hz
raw_ica_fit1 = raw.copy()
raw_ica_fit1.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

## 4.1 Annotate bad ICs from ICA1 for mastoids
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map

In [ ]:
figs_ica_sources_mastoids = ica1.plot_sources(
    title=fname_stem + " | ICA1 sources Mastoids | " + experimenter, 
    show=True,
    inst=raw_ica_fit1
)

In [ ]:
%matplotlib inline
figs_ica_comp_mastoids = ica1.plot_components(
    title=fname_stem + " | ICA1 components Mastoids | " + experimenter,
    show=True,
    inst=raw_ica_fit1
)

In [ ]:
%matplotlib qt
figs_ica_comp_mastoids = ica1.plot_components(
    title=fname_stem + " | ICA1 components Mastoids | " + experimenter,
    show=True,
    inst=raw_ica_fit1
)

In [ ]:
if not PAST_REVIEWER:
    ica_folder = derivatives_folder / fname_stem / "plots" / "ica"
    timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
    for i in range(len(figs_ica_comp_mastoids)):
        save_path = os.path.join(
            ica_folder,
            "allComponents_ICA1_" + experimenter + "_" + str(i+1) + "_" + timestampStr + ".svg")
        figs_ica_comp_mastoids[i].savefig(save_path, transparent=True)

In [ ]:
#save ICA1 here instead?

In [ ]:
#%% Clean the other channels
# The first step is to prepare the raw object for an ICA, and for suggestions
# from ICLabel. The steps are very similar to the previous ones.
del raw_ica_fit1
raw.drop_channels(["M1", "M2"])

In [ ]:
# filter
raw_ica_fit2 = raw.copy()
raw_ica_fit2.filter(
    l_freq=1.0,
    h_freq=100.0,  # Note the higher frequency
    picks=["eeg"],
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

In [ ]:
# change the reference to a common average reference (CAR)
raw_ica_fit2.set_montage(None)
raw_ica_fit2.add_reference_channels(ref_channels="CPz")
raw_ica_fit2.set_montage("standard_1020")
raw_ica_fit2.set_eeg_reference("average", projection=False)
# Note that the CAR is excluding the bad channels.

## 4.2 Annotate bad ICs from ICA2 for EEG
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map


In [ ]:
# Visual inspection
figs_ica_sources = ica2.plot_sources(
    title=fname_stem + " | ICA2 sources | " + experimenter, 
    show=True, 
    inst=raw_ica_fit2
)

In [ ]:
%matplotlib inline
figs_ica_comp = ica2.plot_components(
    title=fname_stem + " | ICA2 components | " + experimenter,
    show=True,
    inst=raw_ica_fit2
)

In [ ]:
%matplotlib qt
figs_ica_comp = ica2.plot_components(
    title=fname_stem + " | ICA2 components | " + experimenter,
    show=True,
    inst=raw_ica_fit2
)

In [ ]:
if not PAST_REVIEWER:
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i in range(len(figs_ica_comp)):
    save_path =  os.path.join(
        ica_folder, 
        "allComponents_ICA2_" + experimenter + "_" + str(i+1) + "_" + timestampStr + ".svg")
    figs_ica_comp[i].savefig(save_path, transparent = True)

## 4.3 Save derivatives

The ICA decomposition can be saved.

In [ ]:
if not PAST_REVIEWER:
    fname_ica1 = derivatives_folder / fname_stem / (fname_stem
                                                    + "_step4_reviewed-1st-"
                                                    + experimenter
                                                    + "-ica.fif")
    fname_ica2 = derivatives_folder / fname_stem / (fname_stem 
                                                    + "_step4_reviewed-2nd-"
                                                    + experimenter
                                                    + "-ica.fif"")

    ica1.save(fname_ica1, overwrite=False)
    ica2.save(fname_ica2, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway